In [1]:
from cmdstanpy import CmdStanModel

import arviz as az
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import FloatProgress
from IPython.display import display
from tqdm import tqdm_notebook
import pickle

from tensorflow_probability.substrates import numpy as tfp
tfd = tfp.distributions

In [2]:
df = pd.read_csv("Py_Dataset.csv")
staz = pd.read_csv("Stazioni_Emilia.csv")
dummies = pd.get_dummies(df.Tipo)

In [90]:
normal_cluster = """
data {
    int<lower=0> N; 
    int<lower=0> p;
    int<lower=0> G;
    int<lower=0> C;
    
    vector[N] Y;
    matrix[N, p] X;
    vector[N] t;
    int stazione[N];
    real omega;
    row_vector[2] coord[G];
}

parameters {
    
    real<lower=0> sigma_sq;
    
    vector[p] beta;
    
    matrix[G, 2] A;
    
    vector[G] w;
    real<lower=0> rho;
    real<lower=0> alpha;
    
    
    vector<lower=0>[C] sigma_l_sq;
    matrix[C,2] mu_l;
    vector<lower=0, upper=1>[C-1] v;
    
    
}


transformed parameters {
    real<lower=0> sigma;
    sigma = sqrt(sigma_sq);
    
    vector<lower=0>[C] sigma_l;
    sigma_l = sqrt(sigma_l_sq);
    
    vector[N] ft;
    ft = to_vector(rep_array(0, N));
    ft[1:N] += (  rows_dot_product(A[stazione[1:N],1],sin(omega*t[1:N])) 
                + rows_dot_product(A[stazione[1:N],2],cos(omega*t[1:N]))  );

    vector[N] mu;
    mu = ft + X*beta ;
    mu[1:N] += w[stazione[1:N]];
         
    cov_matrix[G] H = cov_exp_quad(coord, alpha, rho);
    
    vector<lower=0, upper=1> [C-1] cumprod_one_minus_v;
    cumprod_one_minus_v = exp(cumulative_sum(log1m(v)));
    simplex[C] eta;
    eta[1] = v[1];
    eta[2:(C-1)] = v[2:(C-1)] .*cumprod_one_minus_v[1:(C-2)];
    eta[C] = cumprod_one_minus_v[C - 1];
    
    
    real param=2.0;
}


model {  

    sigma_sq ~ inv_gamma(3,2);
    
    beta ~ normal([-0.1, -0.1, 0.1], 1);
    
    rho ~ beta(10, 2000);
    
    w ~ multi_normal(rep_vector(0,G), H);
    
    Y ~ normal(mu, sigma);  
    
    
    
    
    for (i in 1:C){
         mu_l[i, :] ~ multi_normal(rep_vector(0,2), diag_matrix(rep_vector(2,2)));
         }
         
    sigma_l_sq ~ inv_gamma(10,9);
    
    v ~ beta(1, param);
    
    for (g in 1:G){
        vector[C] lps = log(eta);
        for (k in 1:C){
            lps[k] += normal_lpdf(A[g, :] | mu_l[k,:], sigma_l[k]);
            }
        target += log_sum_exp(lps);
        }
}


generated quantities  {
  vector[N] log_lik;
  for (j in 1:N) {
    log_lik[j] = normal_lpdf(Y[j] | mu[j], sigma);
  }
  
  
  int cluster_allocs[G];
  for(g in 1:G){
  vector[C] log_probs = log(eta);
  for (k in 1:C){
     log_probs[k] += normal_lpdf(A[g, :] | mu_l[k,:], sigma_l[k]);
     }
  cluster_allocs[g] = categorical_rng(softmax(log_probs));
  }
  
}
"""

stan_file = "./normal_cluster.stan"

with open(stan_file, "w") as f:
    print(normal_cluster, file=f)

normal_cluster = CmdStanModel(stan_file=stan_file)


INFO:cmdstanpy:compiling stan program, exe file: /Users/michelafrigeri/JupyterProjects/PROGETTO/normal_cluster
INFO:cmdstanpy:compiler options: stanc_options={}, cpp_options={}
Warning in '/Users/michelafrigeri/JupyterProjects/PROGETTO/normal_cluster.stan', line 53, column 22: cov_exp_quad is deprecated and will be removed in the future. Use gp_exp_quad_cov instead.
INFO:cmdstanpy:compiled model file: /Users/michelafrigeri/JupyterProjects/PROGETTO/normal_cluster


In [91]:
y = np.array(df.Y_log)
x = np.matrix( pd.concat([df.quota, dummies.Fondo, dummies.Industriale], axis=1) )
t = np.array(df.Time)
r = np.array(df.Rural)
stazione = np.array(df.id)
omega = 2*np.pi/365
coord = np.matrix( pd.concat([staz.Lat, staz.Long], axis=1) )

N = len(y)
p = 3   # Quota + Tipo(2 dummies)
G = 49  # numero stazioni in Emilia-Romagna
C = 10  # numero cluster

In [92]:
reg_data = {
    "N": N,   # 18 000 circa (49x365)
    "p": p,   # 4
    "G": G,
    "C": C, # prova con 20
    
    "Y": y,
    "X": x, 
    "t": t,
    "stazione": stazione,
    "omega": omega,
    "coord": coord
}

fit_clust = normal_cluster.sample(data=reg_data, chains=2, parallel_chains=2, 
                             iter_warmup=30, iter_sampling=50, show_progress=True)



Chain 2 -   done: 100%|█████████████████████████| 80/80 [03:29<00:00,  2.62s/it]


In [95]:
# cluster allocation
output_clust = fit_clust.stan_variable('cluster_allocs')
output_clust.shape

(100, 49)

In [96]:
# prima iterazione : 1st chain
output_clust[0,:]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1.])

In [85]:
# ultima iterazione : 1st chain
output_clust[49,:] 

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [98]:
# prima iterazione : 2nd chain
output_clust[50,:]

array([4., 1., 1., 5., 3., 7., 1., 3., 3., 9., 3., 4., 9., 1., 4., 7., 3.,
       3., 4., 3., 4., 1., 2., 7., 3., 3., 2., 3., 3., 3., 1., 3., 3., 3.,
       1., 3., 5., 4., 3., 6., 1., 3., 1., 1., 7., 1., 3., 4., 7.])

In [99]:
# ultima iterazione : 2nd chain
output_clust[99,:] 

array([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.])

In [105]:
def get_psm(clus_alloc_chain):
    """
    Returns the posterior similarity matrix, i.e.
        out[i, j] = P(c_i == c_j)
    for each pair of observations
    """
    c_chain = np.vstack(clus_alloc_chain)
    out = np.zeros((c_chain.shape[1], c_chain.shape[1]))
    for i in range(c_chain.shape[1]):
        for j in range(i):
            out[i, j] = np.mean(c_chain[:, i] == c_chain[:, j])
            
    return out + out.T + np.eye(out.shape[0])


def minbinder_sample(clus_alloc_chain, psm):
    losses = np.zeros(len(clus_alloc_chain))
    c_chain = np.vstack(clus_alloc_chain)
    for i in range(c_chain.shape[1]):
        for j in range(i):
            # TODO complete here!
            losses += 2 * (np.mean(c_chain[:, i] == c_chain[:, j]) - psm[i, j])
    best_iter = np.argmin(losses)
    return clus_alloc_chain[best_iter]

In [106]:
psm = get_psm(output_clust)

In [107]:
best_clus = minbinder_sample(output_clust, psm)

In [111]:
A = fit_clust.stan_variable('A')
A = A[:,:,0]
plt.hist(, density=True, alpha=0.3)
for h in range(3):
    currd = data[best_clus == h]
    plt.scatter(currd, np.zeros_like(currd) + 0.001 * (h+1))

In [114]:
best_clus

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1.])